# Workshop: From Simple to Agentic RAG

## Setup


🎉 Free Cohere API key

Before you run this notebook, head over to this [link to redeem a free Cohere API key](https://dashboard.cohere.com/billing?tab=payment&creditCode=WANDB-Y01D-JK8B).

Alternatively if you have a Cohere API key feel free to proceed.

To redeem your workshop credits:
1. Make sure you are the admin of the account/team.
2. Follow your unique link or use your code in the Cohere wallet to redeem the credits.
[Link](https://dashboard.cohere.com/billing?tab=payment&creditCode=WANDB-Y01D-JK8B)

Next head over to [Tavily](https://app.tavily.com/home) and sign up for a free account and get your API key.

We will use [`uv`](https://docs.astral.sh/uv/getting-started/installation/) to install the workshop package and its dependencies.


```bash
!git clone https://github.com/parambharar/workshops.git
!cd workshops/rag
!pip install uv
!uv pip install -e .
```

Create an `.env` file in the workshop/rag directory and add the API keys to it:

```.env
COHERE_API_KEY="YOUR_COHERE_API_KEY"
TAVILY_API_KEY="YOUR_TAVILY_API_KEY"
```


In [1]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2


In [2]:
import json
import wandb
import weave
from copy import deepcopy
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from utils import convert_contents_to_text, make_id, render_doc, printmd, chunk_simple, chunk_markdown, load_dataset, chunk_dataset
from retrieval_metrics import RetrievalScorer
from response_metrics import ResponseScorer
from retriever import TfidfSearchEngine, BM25SearchEngine, DenseSearchEngine, Retriever, RetrieverWithReranker, HybridRetrieverWithReranker, VectorStoreSearchEngine
from generation import SimpleResponseGenerator, QueryEnhancedResponseGenerator
from pipeline import SimpleRAGPipeline, QueryEnhancedRAGPipeline
from query_enhancer import QueryEnhancer
from agent import WandbAgent

We'll use W&B Artifacts to track and version data as the inputs and outputs of your W&B Runs. 
For example, a model training run might take in a dataset as input and produce a trained model as output. 
W&B Artifacts is a powerful object storage with rich UI functionalities.

Below we are downloading an artifact named wandb_docs which will download 400 odd files in your `data/docs` directory. This is will act as our data source.


In [ ]:
wandb_api = wandb.Api()
docs_root = wandb_api.artifact("parambharat/rag-workshop/documentation:latest", type="dataset").download(root="data/docs")
docs_file =  f"{docs_root}/wandb_docs.jsonl"
with open(docs_file, "r") as f:
    docs = f.readlines()
docs = [json.loads(doc) for doc in docs]


In [ ]:
weave_client = weave.init("rag-workshop")

## A simple RAG Pipeline

First, we will learn about building a simple RAG pipeline. We will mainly focus on how to preprocess and chunk the data followed by building a simple retrieval engine without using any fancy "Vector Index". The idea is to show the inner working of a retrieval pipeline and make you understand the workflow from a user query to a generated response using an LLM.

In [7]:
document_chunks = []
for doc in docs:
    chunks = chunk_simple(doc["content"], chunk_size=500)
    doc_id = make_id(doc["content"])
    for chunk in chunks:
        doc_chunk = deepcopy(doc)
        doc_chunk["chunk"] = chunk
        doc_chunk["text"] = convert_contents_to_text(chunk)
        doc_chunk["chunk_id"] = make_id(chunk)
        doc_chunk["doc_id"] = doc_id
        document_chunks.append(doc_chunk)

In [8]:
tfidf_search_engine = await TfidfSearchEngine().fit(document_chunks)

class TFIDFRetriever(Retriever):
    pass

tfidf_retriever = TFIDFRetriever(search_engine=tfidf_search_engine)
# retrieved_docs = await tfidf_search_engine.search(query="What are Artifacts?", top_k=5)
# for doc in retrieved_docs:
#     render_doc(doc)

In [9]:
# docs_data = [{"document": item["text"]} for item in retrieved_docs]
simple_response_generator = SimpleResponseGenerator()
# response = await simple_response_generator.invoke(
#     query="What are Artifacts?", documents=docs_data
# )
# printmd(response["choices"][0]["message"]["content"])

In [10]:
class TFIDFRAGPipeline(SimpleRAGPipeline):
    pass
tfidf_rag_pipeline = TFIDFRAGPipeline(
    retriever=tfidf_retriever,
    generator=simple_response_generator
)
# response = await tfidf_rag_pipeline.invoke(query="What are Artifacts?",)
# printmd(response["answer"])

In [11]:
## Evaluating the RAG Pipeline



In [12]:
### Evaluating the Retrieval

In [13]:
# with open("data/eval_dataset.jsonl", "r") as f:
#     eval_dataset = [json.loads(line) for line in f.readlines()]
# eval_dataset = weave.Dataset(name="eval_dataset", description="Evaluation dataset for RAG Pipeline", rows=eval_dataset)
# weave.publish(eval_dataset, name="eval_dataset")
# eval_dataset = weave.ref("weave:///parambharat/rag-workshop/object/eval_dataset:DkS8t0EnHiHQAmdmd9M9UzeJSyiOTNIr924v62abT4Q").get()

In [ ]:
eval_dataset = weave.ref("weave:///parambharat/rag-workshop/object/eval_dataset:9aTli8u1UJBYQD6aBWnYMUpokMjNUrXjQBspuAWBbYE").get()

print("Number of evaluation samples: ", len(eval_dataset.rows))

In [15]:
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=[RetrievalScorer(name="retrieval_scorer", description="Retrieval metrics")],
    preprocess_model_input=lambda x: {"query": x["question"], "top_k": 5},
)

In [ ]:
tfidf_retrieval_scores = await retrieval_evaluation.evaluate(
    model=tfidf_retriever,
    __weave={"display_name": "TFIDF Retrieval"}
)


In [17]:
### Evaluationg the Response Generation

In [18]:


response_evaluation = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=[ResponseScorer(name="response_scorer", description="Response metrics")],
    preprocess_model_input=lambda x: {"query": x["question"]},
)


In [ ]:
tfidf_response_scores = await response_evaluation.evaluate(
    model=tfidf_rag_pipeline,
    __weave={"display_name": "TFIDF RAG Pipeline"}
)

In [ ]:
bm25_search_engine = await BM25SearchEngine().fit(document_chunks)

class BM25Retriever(Retriever):
    pass

bm25_retriever = BM25Retriever(search_engine=bm25_search_engine)

# retrieved_docs = await bm25_retriever.invoke(query="What are Artifacts?", top_k=5)

# for doc in retrieved_docs:
#     render_doc(doc)


In [ ]:
retrieval_scores = await retrieval_evaluation.evaluate(
    model=bm25_retriever,
    __weave={"display_name": "BM25 Retrieval"}
)

In [22]:

class BM25RAGPipeline(SimpleRAGPipeline):
    pass

bm25_rag_pipeline = BM25RAGPipeline(
    retriever=bm25_retriever,
    generator=simple_response_generator
)

# response = await bm25_rag_pipeline.invoke(query="What are Artifacts?",)

# printmd(response["answer"])

In [ ]:
response_scores = await response_evaluation.evaluate(
    model=bm25_rag_pipeline,
    __weave={"display_name": "BM25 RAG Pipeline"}
)

In [24]:
dense_search_engine = DenseSearchEngine()
dense_search_engine = await dense_search_engine.fit(document_chunks)

class DenseRetriever(Retriever):
    pass

dense_retriever = DenseRetriever(search_engine=dense_search_engine)
# retrieved_docs = await dense_retriever.invoke(query="What are Artifacts?", top_k=5)
# for doc in retrieved_docs:
#     render_doc(doc)


In [ ]:
retrieval_scores = await retrieval_evaluation.evaluate(
    model=dense_retriever,
    __weave={"display_name": "Dense Retrieval"}
)


In [26]:
class DenseRAGPipeline(SimpleRAGPipeline):
    pass

dense_rag_pipeline = DenseRAGPipeline(
    retriever=dense_retriever,
    generator=simple_response_generator
)

# response = await dense_rag_pipeline.invoke(query="What are Artifacts?",)

# printmd(response["answer"])

In [ ]:
response_scores = await response_evaluation.evaluate(
    model=dense_rag_pipeline,
    __weave={"display_name": "Dense RAG Pipeline"}
)

In [28]:

class DenseRerankedRetriever(RetrieverWithReranker):
    pass

dense_reranked_retriever = DenseRerankedRetriever(search_engine=dense_search_engine,)

# retrieved_docs = await dense_reranked_retriever.invoke(query="What are Artifacts?", top_k=10, top_n=5)

# for doc in retrieved_docs:
#     render_doc(doc)


In [ ]:
retrieval_scores = await retrieval_evaluation.evaluate(
    model=dense_reranked_retriever,
    __weave={"display_name": "Dense Reranked Retrieval"}
)

In [ ]:
class DenseRerankedRAGPipeline(SimpleRAGPipeline):
    pass

dense_reranked_rag_pipeline = DenseRerankedRAGPipeline(
    retriever=dense_reranked_retriever,
    generator=simple_response_generator
)

# response = await dense_reranked_rag_pipeline.invoke(query="What are Artifacts?",)

# printmd(response["answer"])
response_scores = await response_evaluation.evaluate(
    model=dense_reranked_rag_pipeline,
    __weave={"display_name": "Dense Reranked RAG Pipeline"}
)


In [31]:
hybrid_retriever = HybridRetrieverWithReranker(
    sparse_search_engine=tfidf_search_engine,
    dense_search_engine=dense_search_engine
)

# retrieved_docs = await hybrid_retriever.invoke(query="What are Artifacts?", top_k=10, top_n=5)

# for doc in retrieved_docs:
#     render_doc(doc)

In [ ]:
hybrid_retrieval_scores = await retrieval_evaluation.evaluate(
    model=hybrid_retriever,
    __weave={"display_name": "Hybrid Retrieval"}
)


In [ ]:
class HybridRAGPipeline(SimpleRAGPipeline):
    pass

hybrid_rag_pipeline = HybridRAGPipeline(
    retriever=hybrid_retriever,
    generator=simple_response_generator
)

# response = await hybrid_rag_pipeline.invoke(query="What are Artifacts?",)

# printmd(response["answer"])
hybrid_response_scores = await response_evaluation.evaluate(
    model=hybrid_rag_pipeline,
    __weave={"display_name": "Hybrid RAG Pipeline"}
)


In [34]:
# Structured Chunking

In [ ]:
chunked_docs = chunk_markdown(docs[0]["content"], chunk_size=500)
chunked_docs

In [36]:
document_chunks = []
for doc in docs:
    chunks = chunk_markdown(doc["content"], chunk_size=500)
    doc_id = make_id(doc["content"])
    for chunk in chunks:
        doc_chunk = deepcopy(doc)
        doc_chunk["chunk"] = chunk
        doc_chunk["text"] = convert_contents_to_text(chunk)
        doc_chunk["chunk_id"] = make_id(chunk)
        doc_chunk["doc_id"] = doc_id
        document_chunks.append(doc_chunk)

In [37]:
bm25_search_engine = await BM25SearchEngine().fit(document_chunks)
dense_search_engine = await DenseSearchEngine().fit(document_chunks)
hybrid_retriever = HybridRetrieverWithReranker(
    sparse_search_engine=bm25_search_engine,
    dense_search_engine=dense_search_engine
)

# retrieved_docs = await hybrid_retriever.invoke(query="What are Artifacts?", top_k=10, top_n=5)

# for doc in retrieved_docs:
#     render_doc(doc)


In [ ]:
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=[RetrievalScorer(name="retrieval_scorer", description="Retrieval metrics")],
    preprocess_model_input=lambda x: {"query": x["question"], "top_k": 10, "top_n": 5},
)
hybrid_retrieval_scores = await retrieval_evaluation.evaluate(
    model=hybrid_retriever,
    __weave={"display_name": "Hybrid Retrieval With Structured Chunking"}
)

In [39]:

class HybridRAGPipeline(SimpleRAGPipeline):
    pass

hybrid_rag_pipeline = HybridRAGPipeline(
    retriever=hybrid_retriever,
    generator=simple_response_generator)

# response = await hybrid_rag_pipeline.invoke(query="What are Artifacts?",)

# printmd(response["answer"])

In [ ]:
hybrid_response_scores = await response_evaluation.evaluate(
    model=hybrid_rag_pipeline,
    __weave={"display_name": "Hybrid RAG Pipeline With Structured Chunking"}
)
hybrid_response_scores


In [41]:
## More data, vector store

In [42]:
# full_dataset = load_dataset(docs_root)
# chunked_dataset = chunk_dataset(full_dataset, chunk_size=500)
# vectorstore_search_engine = VectorStoreSearchEngine()
# vectorstore_search_engine = await vectorstore_search_engine.fit(chunked_dataset)
vectorstore_search_engine = VectorStoreSearchEngine()
vectorstore_search_engine = await vectorstore_search_engine.load()

In [43]:
# results = await vectorstore_search_engine.search(
#     query="What are Artifacts?",
#     top_k=5,
#     filters="file_type in ('notebook', 'markdown')")
# for doc in results:
#     render_doc(doc)

In [44]:
class VectorStoreRetriever(RetrieverWithReranker):
    pass

vectorstore_retriever = VectorStoreRetriever(search_engine=vectorstore_search_engine)

# results = await vectorstore_retriever.invoke(query="What are Artifacts?", top_k=10, top_n=5, filters="file_type in ('notebook', 'markdown')")
# for doc in results:
#     render_doc(doc)

In [45]:
## Query Enhancement

In [ ]:
query_enhancer = QueryEnhancer()
results = await query_enhancer.invoke("What are W&B Artifacts?")
results

In [ ]:
query_enhanced_rag_pipeline = QueryEnhancedRAGPipeline(
    query_enhancer=query_enhancer,
    retriever=vectorstore_retriever,
    response_generator=QueryEnhancedResponseGenerator()
)
response = await query_enhanced_rag_pipeline.invoke("What are Artifacts?")


In [ ]:
query_enhanced_response_scores = await response_evaluation.evaluate(
    model=query_enhanced_rag_pipeline,
    __weave={"display_name": "Query Enhanced RAG Pipeline"}
)
query_enhanced_response_scores


In [ ]:
## Agentic RAG

In [ ]:
agent = WandbAgent()

query = "Are there any notebooks related to artifacts?"
answer = await agent.invoke(query)
printmd(answer["answer"])


In [ ]:
query = "Where can I find the output of a run?"
answer = await agent.invoke(query)
printmd(answer["answer"])


In [ ]:
query = "Who are the authors of the sentence BERT paper?"
answer = await agent.invoke(query)
printmd(answer["answer"])

In [ ]:
query = "Explain what is a W&B Run and how do I view a specific run"
answer = await agent.invoke(query)
printmd(answer["answer"])